In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob

In [2]:
# #SET GROUPS FOR THE CORE-SHELLS
# group lithium type 1
# group coreshell type 2 3 4 5 6 7 8 9
# group cores type  2 4 6 8
# group shells type 3 5 7 9

# #SET BOND_STYLE UNDER PAIR STYLE/COEFF
# bond_style harmonic
# bond_coeff 1  93.7     0.0
# bond_coeff 2  196.3    0.0
# bond_coeff 3  95.0     0.0
# bond_coeff 4  65.0     0.0

# #ADDITIONAL COMPUTE COMMANDS????
# compute         cs_temp all temp/cs cores shells             #computes temperature of the coreshells
# compute         cs_press all pressure cs_temp                #computes pressure based on coreshell temp
# velocity        all create ${T1} 100 bias yes temp cs_temp   #velocity not including relative motion but only centre mass of cs pair
# velocity        all scale ${T1} temp cs_temp                 #velocity scaled by temp include cs modification


In [3]:
#CHANGE LAST ELEMENT???
elements = pmg.Element('Li'), pmg.Element('Ni'), pmg.Element('O'), pmg.Element('Ge')
li, ni, o, ge = elements

pairs = [(li,ge), (ni,ge), (ge,ge)]

charges = {li: 1, ni: 3, o: 0.960, ge: -2.960}
masses = {li: 6.9410, ni: 52.8237, o: 14.3991, ge: 1.5999}

bpp_def = {'Li_Ge_A'   : 456.783,
           'Li_Ge_rho' : 0.212,
           'Li_Ge_C'   : 0.0,
           'Ni_Ge_A'   : 1540.922,
           'Ni_Ge_rho' : 0.323,
           'Ni_Ge_C'   : 0.425,
           'Ge_Ge_A'   : 22764.7000,
           'Ge_Ge_rho' : 0.1490,
           'Ge_Ge_C'   : 20.4}

bpp = {'Li_Ge_A'   : 456.783,
       'Li_Ge_rho' : 0.212,
       'Li_Ge_C'   : 0.0,
       'Ni_Ge_A'   : 1540.922,
       'Ni_Ge_rho' : 0.323,
       'Ni_Ge_C'   : 0.425,
       'Ge_Ge_A'   : 22764.7000,
       'Ge_Ge_rho' : 0.1490,
       'Ge_Ge_C'   : 20.4}


In [4]:
structures = []
for structure in glob.glob('poscars/POSCAR*'):
    st = pmg.Structure.from_file(structure)
    structures.append(st)

In [6]:
for structure in structures:
    new_sites = []
    [new_sites.append(structure.frac_coords[i]) for i, species in enumerate(structure.species) if species.symbol == "O" ]
    for i, species in enumerate(new_sites):
        structure.append('Ge', new_sites[i], validate_proximity=False, properties=None)

In [24]:
def init(structure):
    lmp = lammps.Lammps(units='metal',
                        style = 'full',
                        args=['-log', 'none', '-screen', 'none'])
    lmp.system.add_pymatgen_structure(structure, elements)
    lmp.command('kspace_style pppm 1e-6')
    lmp.command('pair_style buck/coul/long 12.0')
    lmp.command('pair_coeff * * 0 1 0')
    
        
    #ADDED FOR CORESHELL
    lmp.command('group cores type 3')
    lmp.command('group shells type 4')

    

    for c in charges:
        lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))
    for i, m in enumerate(masses):
        lmp.command('set atom %d mass %f' % (elements.index(m)+1, masses[m]))
    
    return lmp

In [25]:
def update_potentials(**kwargs):
    bpp.update(kwargs)
    
def set_potentials(instance):
    for pair in pairs:

        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]), 
                                      dex(pair[1]),  
                                      bpp['{}_{}_A'.format(pair[0], pair[1])], 
                                      bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                      bpp['{}_{}_C'.format(pair[0], pair[1])]
                                     )
             )
#     lmp.command('bond_style harmonic')
#     lmp.command('bond_coeff 1 65.0 0.0')

def dex(elin): return elements.index(elin) + 1

In [26]:
def simfunc(**kwargs):
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([instances[0].system.total,3, len(instances)])
        
        for instance in instances:
            set_potentials(instance)
            instance.run(0)
            out[:,:,instances.index(instance)] = instance.system.forces
            
    else: out = np.ones([instances[0].system.total,3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
        
    return out

In [27]:
instances = [init(structure) for structure in structures]

LammpsNormalError: ERROR: Cannot set this attribute for this atom style (../set.cpp:179)


In [ ]:
expected = np.zeros([instances[0].system.total,3, len(instances)])

In [ ]:
with pm.Model() as model:
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  100)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
    Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.05)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=1), draws=500)

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')